*The goal of this project is to predict the violent crime rate based upon features of a community.  The Communities and Crime data set is available at  https://archive.ics.uci.edu/dataset/183/communities+and+crime .*

##Step 1: Import the data.

In [472]:
pip install ucimlrepo

In [473]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

#fetch data
communities_and_crime = fetch_ucirepo(id=183)

# data (as pandas dataframes)
X = communities_and_crime.data.features
y = communities_and_crime.data.targets

info = [communities_and_crime.metadata, communities_and_crime.variables]

##Step 2: Data preparation.

In [474]:
#Check the response variable - Violent Crimes per Population for missing or non float values.
#If unclean this should prompt error.
check = lambda x: True if type(x) == float else False
assert y['ViolentCrimesPerPop'].apply(check).sum() == y.shape[0], 'Error y contains non-float entries'

In [475]:
df = X.replace('?',np.nan) #replacing the "?" entries with np.nan values
print(f'There are {df.shape[1]} explanatory variables each containting {df.shape[0]} entries\n')

df.head(3)

There are 127 explanatory variables each containting 1994 entries



,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,8,NaN,NaN,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14
1,53,NaN,NaN,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,NaN,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.00,NaN
2,24,NaN,NaN,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,NaN,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.00,NaN


Now let us extract the explanatory variables that contain no missing values.

In [476]:
bool_musk = (df.isna().sum() == 0).values
clean_col = list(df.columns[bool_musk])

#Check that they are indeed clean.
for col in clean_col:
  assert df[col].isna().sum() == 0, 'ERROR'

print(f'Out of the {df.shape[1]} there are {len(clean_col)} explanatory variables with no missing values.\nTherefore, {(len(clean_col)/df.shape[1])*100:.2f}% of the explanatory variables are clean; these will be included in our analysis.')

Out of the 127 there are 102 explanatory variables with no missing values.
Therefore, 80.31% of the explanatory variables are clean; these will be included in our analysis.


Our filter of explanatory variables was very naive (if there were missing values we dropped that variable).  Lets go back and check the percentage of data missing in each of the unclean variables.  Furthermore, let us check if any of these variables are such that we would want to include them, e.g. a variable such as *proportion of the community that have a criminal record* should not be easily discarded as this could very well be incredibly significant to the analysis.

In [477]:
unclean_col = list(df.columns[~bool_musk])

freq_na = df.isna().sum()[unclean_col]

dict_na = {var:[] for var in unclean_col}

for key, val in dict(freq_na).items():
  dict_na[key].append(freq_na[key])
  dict_na[key].append(float(f'{(val / df.shape[0])*100:.3f}'))

df_na = pd.DataFrame(dict_na).T.rename(columns = {0:'NA Count',1:'NA (%)'}).sort_values(by = 'NA (%)')
df_na

,NA Count,NA (%)
OtherPerCap,1.0,0.050
county,1174.0,58.877
community,1177.0,59.027
LemasPctPolicOnPatr,1675.0,84.002
PolicOperBudg,1675.0,84.002
PolicCars,1675.0,84.002
PolicAveOTWorked,1675.0,84.002
NumKindsDrugsSeiz,1675.0,84.002
OfficAssgnDrugUnits,1675.0,84.002
PctPolicMinor,1675.0,84.002


Clearly we should not have dropped *OtherPerCap* as it only has 1 missing entry out of 1994 entries = 5 bps.  This is negligible.  Furthermore, the other variables have high missing data (>50%) and should be dropped.


In [478]:
final_vars = clean_col + ['OtherPerCap']
print(f'We have {len(final_vars)} explanatory variables to use.\n')
#Note 'OtherPerCap has 1 missing value.
#I will take the mean of OtherPerCap and replace the missing value with this.

bool_musk2 = df['OtherPerCap'].isna()
na_index = df['OtherPerCap'].loc[bool_musk2].index[0] #is 130

mean_OPC = ((df['OtherPerCap'].loc[df['OtherPerCap'] != np.nan]).apply(lambda x: float(x))).mean()
median_OPC = ((df['OtherPerCap'].loc[df['OtherPerCap'] != np.nan]).apply(lambda x: float(x))).median()

print(f'The mean of OtherPerCap: {mean_OPC}\nThe median of OtherPerCap: {median_OPC}\n\nUsing the mean to approximate the missing value is viable.')

df.loc[na_index, 'OtherPerCap'] = round(mean_OPC, 4)

We have 103 explanatory variables to use.

The mean of OtherPerCap: 0.2847415955845459
The median of OtherPerCap: 0.25

Using the mean to approximate the missing value is viable.


Now we drop our variables with high percentages of missing data and perform the regression using the final 103 explanatory variables in the list *final_vars*.


In [479]:
X_clean = df[final_vars]
X_clean.head()

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,8,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,53,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,24,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,34,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,42,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51


Now to make sure all data is of the correct type, i.e., the strings containing numbers are floats we run the following code chunk.

In [480]:
for var in final_vars:
    if X_clean[var].dtype == 'O':  # Check if the column is an object type (potentially non-numeric)
        try:
            X_clean.loc[:, var] = X_clean[var].astype(float)
        except ValueError:
            print(f"Column {var} contains non-numeric values and cannot be converted.")
    else:
        X_clean.loc[:, var] = X_clean[var].astype(float)


Column communityname contains non-numeric values and cannot be converted.


In [481]:
X_clean.head()

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,8,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,53,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,24,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,34,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,42,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51


Now to apply one hot encoding on the categorical features: [*state*, *communityname*].  This requires creating two matrix of zeros with columns being the *state* values.  This matrix will be very sparse.  The next step is to populate the matrix with a 1 if the *state* is contained in row i for integer i between 0 and 1994.  The same process is repeated for *communityname*.

In [482]:
#creating and populating state matrix
unique_states = list(X_clean['state'].apply(lambda x: int(x)).unique())
states_df = np.zeros(shape=(df.shape[0],len(unique_states)))
states_df = pd.DataFrame(states_df, columns=unique_states)

for index, value in enumerate(X_clean['state']):
  states_df.loc[index,value] = 1

assert states_df.sum().sum() == X_clean['state'].shape[0], 'Error in populating matrix'

#creating and populating communityname matrix
unique_communitynames = X_clean['communityname'].unique()
communitynames_df = np.zeros(shape=(df.shape[0],len(unique_communitynames)))
communitynames_df = pd.DataFrame(communitynames_df, columns=unique_communitynames)

for index, value in enumerate(X_clean['communityname']):
  communitynames_df.loc[index,value] = 1

assert communitynames_df.sum().sum() == X_clean['communityname'].shape[0], 'Error in populating matrix'

In [483]:
states_df

,8,53,24,34,42,6,44,21,29,36,...,13,23,38,27,20,56,2,11,32,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [484]:
communitynames_df

,Lakewoodcity,Tukwilacity,Aberdeentown,Willingborotownship,Bethlehemtownship,SouthPasadenacity,Lincolntown,Selmacity,Hendersoncity,Claytoncity,...,SpringGardentownship,Maumeecity,AlexanderCitycity,Southingtontown,Burrillvilletown,TempleTerracecity,Seasidecity,Waterburytown,Walthamcity,Ontariocity
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Now we will drop the columns *state* and *communityname* from dataframe *X_clean* and concat dataframes *states_df* and *communitynames_df* to *X_clean*.

In [485]:
X_clean = X_clean.drop(['state','communityname'],axis=1, errors='ignore')
X_combined1 = pd.concat([X_clean, states_df], axis=1)
X_combined = pd.concat([X_combined1, communitynames_df], axis=1)
assert X_combined.shape[1] == X_clean.shape[1] + states_df.shape[1] + communitynames_df.shape[1], 'Error in concatenation of dataframes'
X_final = X_combined

We have wrangled our final dataframe (*X_final*) for the explanatory varaiables.  We are now ready to do some analysis -> regressing the violent crime rate based upon features of a community.

In [486]:
X_final

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,SpringGardentownship,Maumeecity,AlexanderCitycity,Southingtontown,Burrillvilletown,TempleTerracecity,Seasidecity,Waterburytown,Walthamcity,Ontariocity
0,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,10,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1990,10,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1991,10,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1992,10,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
